In [2]:
#import common librarie
from zipfile import ZipFile 
import os.path
from os import path
import pandas as pd
import numpy as np
import base64
from itertools import groupby
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sns



In [3]:
#import libraries to handle classic chinese word
from jiayan import load_lm
from jiayan import CharHMMTokenizer

# Baseline Model

## Load Data

In [4]:
file_name = "poems_with_new_tags.zip"
poem_df = pd.read_csv('poems_with_new_tags.zip', compression='zip', header=0, quotechar='"')
poem_df.head()

,content,dynasty,author,tags,star,author_stars,title,tags_list,new_tags,new_first_tag,写物,劝勉,家庭,快乐,悲苦,政治,朋友,游玩
0,《吴都赋》云：“户藏烟浦，家具画船。”唯吴兴为然。春游之盛，西湖未能过也。己酉岁，予与萧时父...,宋代,姜夔,春游;怀人;,64,279,琵琶仙·《吴都赋》云：「户藏烟浦,"['春游', '怀人']",政治;游玩,政治,0,0,0,0,0,1,0,1
1,《廿一史弹词》第三段说秦汉开场词滚滚长江东逝水，浪花淘尽英雄。是非成败转头空。青山依旧在，几...,明代,杨慎,咏史;抒怀;人生;哲理,3244,131,临江仙·滚滚长江东逝水,"['咏史', '抒怀', '人生', '哲理']",写物;劝勉;悲苦;政治,写物,1,1,0,0,1,1,0,0
2,《水经》云：“彭蠡之口有石钟山焉。”郦元以为下临深潭，微风鼓浪，水石相搏，声如洪钟。是说也，...,宋代,苏轼,古文观止;纪游;写景;写山,306,4011,石钟山记,"['古文观止', '纪游', '写景', '写山']",写物;政治;游玩,写物,1,0,0,0,0,1,0,1
3,【序】辛亥之冬，予载雪诣石湖。止既月，授简索句，且征新声，作此两曲。石湖把玩不已，使工妓隶习...,宋代,姜夔,咏物;梅花,74,279,暗香疏影,"['咏物', '梅花']",劝勉,劝勉,0,1,0,0,0,0,0,0
4,〔一枝花〕 攀出墙朵朵花，折临路枝枝柳。花攀红蕊嫩，柳折翠条柔，浪子风流。凭着我折柳攀花手...,元代,关汉卿,;散曲;抒情;生活,140,124,【南吕】一枝花不伏老,"['散曲', '抒情', '生活']",写物;快乐;悲苦,写物,1,0,0,1,1,0,0,0


## Split Train and Test Data

In [5]:
poems_info = poem_df[['content','dynasty','author','title']]
tags = poem_df.drop(['content','dynasty','author','title','star','author_stars'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(poems_info, tags, test_size=0.33)

In [6]:
print(len(X_train)==len(y_train))

True


In [7]:
X_train.head()

,content,dynasty,author,title
3564,流水麹尘，艳阳醅酒，画舸游情如雾。笑拈芳草不知名，乍凌波、断桥西堍。垂杨漫舞。总不解、将春系...,宋代,吴文英,西子妆慢·湖上清明薄游
4389,竹窗听雨，坐久隐几就睡，既觉，见水仙娟娟于灯影中窗外捎溪雨响。映窗里、嚼花灯冷。浑似萧湘系孤...,宋代,吴文英,夜游宫·竹窗听雨
5753,雨霁巫山上，云轻映碧天。远风吹散又相连，十二晚峰前。暗湿啼猿树，高笼过客船。朝朝暮暮楚江边，...,五代,毛文锡,巫山一段云·雨霁巫山上
2749,春草细还生，春雏养渐成。茸茸毛色起，应解自呼名。,元代,揭傒斯,画鸭
918,前有毒蛇后猛虎，溪行尽日无村坞。江风萧萧云拂地，山木惨惨天欲雨。女病妻忧归意速，秋花锦石谁复...,唐代,杜甫,发阆中


In [8]:
y_train.head()

,tags,tags_list,new_tags,new_first_tag,写物,劝勉,家庭,快乐,悲苦,政治,朋友,游玩
3564,寒食节;感叹;岁月,"['寒食节', '感叹', '岁月']",写物;悲苦;游玩,写物,1,0,0,0,1,0,0,1
4389,雨夜;咏物;写花;抒怀,"['雨夜', '咏物', '写花', '抒怀']",写物;劝勉,写物,1,1,0,0,0,0,0,0
5753,山峰;写景;哀怨,"['山峰', '写景', '哀怨']",家庭;悲苦;政治,家庭,0,0,1,0,1,1,0,0
2749,古诗三百首;题画,"['古诗三百首', '题画']",写物,写物,1,0,0,0,0,0,0,0
918,写景;抒情,"['写景', '抒情']",悲苦;政治,悲苦,0,0,0,0,1,1,0,0


## Tokenizing

In [9]:
lm = load_lm('jiayan.klm')

In [10]:
from jiayan import WordNgramTokenizer

In [11]:
#Naive Bayes
tokenizer = CharHMMTokenizer(lm)
# print(list(tokenizer.tokenize(text)))


def tokenize_poem(some_list, f):
    result = []
    for i, x in enumerate(some_list):
        result.append(f(x))
    return result
X_train_tokenized = tokenize_poem(X_train['content'], lambda x: " ".join(list(tokenizer.tokenize(x))))
print("get there")
X_test_tokenized = tokenize_poem(X_test['content'], lambda x: " ".join(list(tokenizer.tokenize(x))))
count_vect = CountVectorizer()


get there


In [12]:
Encoder = LabelEncoder()
y_train_encoded = Encoder.fit_transform(y_train['new_first_tag'])
y_test_encoded = Encoder.fit_transform(y_test['new_first_tag'])

In [13]:
X_train_tokenized[0]

'流水 麹尘 ， 艳阳 醅酒 ， 画舸 游 情 如雾 。 笑 拈 芳草 不 知 名 ， 乍 凌波 、 断桥西堍 。 垂杨 漫 舞 。 总 不 解 、 将 春 系 住 。 燕 归来 ， 问 彩绳 纤手 ， 如今 何 许 。 欢盟 误 。 一 箭 流光 ， 又 趁 寒食 去 。 不 堪 衰鬓 著 飞花 ， 傍 绿阴 、 冷 烟深 树 。 玄都 秀句 。 记 前 度 、 刘郎 曾 赋 。 最 伤心 、 一片 孤山 细雨 。'

In [14]:
X_train_cv = count_vect.fit_transform(X_train_tokenized)
X_test_cv = count_vect.transform(X_test_tokenized)

In [15]:
# Tfidf_vect = TfidfVectorizer(max_features=5000)
# Tfidf_vect.fit(Corpus['text_final'])
# Train_X_Tfidf = Tfidf_vect.transform(Train_X)
# Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [16]:
y_train_final = y_train.drop(['tags','tags_list','new_tags','new_first_tag'], axis=1)
y_test_final = y_test.drop(['tags','tags_list','new_tags','new_first_tag'], axis=1)

## Other Models

In [22]:
from numpy import mean
from numpy import std
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate

import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt

In [27]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train_tokenized)

X_train = tokenizer.texts_to_sequences(X_train_tokenized)
X_test = tokenizer.texts_to_sequences(X_test_tokenized)

vocab_size = len(tokenizer.word_index) + 1

maxlen = 200

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)


In [30]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()

glove_file = open('/content/drive/My Drive/Colab Datasets/glove.6B.100d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions
glove_file.close()

embedding_matrix = zeros((vocab_size, 100))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

4091

In [32]:
from bert_embedding import BertEmbedding


In [ ]:
deep_inputs = Input(shape=(maxlen,))
embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], trainable=False)(deep_inputs)
LSTM_Layer_1 = LSTM(128)(embedding_layer)
dense_layer_1 = Dense(6, activation='sigmoid')(LSTM_Layer_1)
model = Model(inputs=deep_inputs, outputs=dense_layer_1)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
